# 0 Gold data transform code

reads in gold analysis .mat which is unreadable by matlab because the field names start with numbers

then add the gold analysis on top of the original "all_vals.mat" under GoldReplication folder

save the final result as "add_vals_with_ppm.mat"

In [1]:
import scipy.io as sio
import numpy as np
import re
import os
from mido import MidiFile

In [73]:
input_file = 'Gold_trained_on_mixed2.mat'
output_file = 'Gold_mDW_IC_Entropy.mat'
stimuli_folder = '../../stimuli/Gold'

In [71]:
gold_original = sio.loadmat(input_file)
all_vals = sio.loadmat(all_vals_file)

In [72]:
table = all_vals['all_vals']

KeyError: 'all_vals'

# Get IC/Entropy data

In [18]:
gold_processed = {}
for key in gold_original.keys():
    if key == 'info' or key.startswith('__'):
        continue
    # match numbers and sort
    match = re.search(r'\d+', key)
    if match:
        number = int(match.group())
        field_data = gold_original[key]
        gold_processed[number] = (field_data[0], field_data[1])
    else:
        print(f'{key} does not start with a number.')

gold_sorted = sorted(gold_processed.items())

In [19]:
gold_sorted

[(1,
  (array([3.90966337, 5.2631707 , 3.65430449, 6.08084357, 4.00117571,
          2.90755425, 5.40176248, 5.32113163, 4.95992747, 5.48978797,
          4.59243397, 5.48546814, 4.87295982, 4.31815948, 3.1461139 ,
          1.64122732, 3.45428916, 4.44098553, 2.74916258, 0.89085583,
          0.82158491, 0.62972058, 0.4126499 , 4.09938939, 9.20080039,
          7.08583917, 1.95306697, 1.90911998, 1.12376989, 0.5852237 ,
          0.47926433, 0.33494007, 0.62869258, 0.3292675 , 0.62826706,
          0.50812703, 0.29873996, 0.4110987 , 0.36433825, 0.69226286,
          0.35301679, 0.36299416, 0.60879929, 3.84648743, 5.72169286,
          4.12694357, 2.98271022, 5.74578513, 8.99404168, 1.52253692,
          3.10668298, 0.17287239, 0.48199643, 0.25963623, 0.56114592,
          0.87074292, 0.50927592, 0.33026864, 0.4180697 , 0.38024469,
          0.33866147, 0.33745408, 0.13777175, 0.32471469, 0.36841689,
          0.3745543 , 0.44486075, 0.67641932, 0.52103011, 0.28427699,
          0.527

# Get note length

In [38]:
def get_note_lengths(midi_file):
    mid = MidiFile(midi_file)
    note_lengths = []
    active_notes = {}
    for track in mid.tracks:
        time = 0
        for msg in track:
            if msg.type == 'note_on':
                if msg.velocity > 0:
                    active_notes[msg.note] = time
                # off note, velocity = 0
                else:
                    if msg.note in active_notes:
                        start_time = active_notes.pop(msg.note)
                        note_lengths.append(time - start_time)
            time += msg.time
    
    print(f"Found {len(note_lengths)} notes in {midi_file}")
    return np.array(note_lengths)

In [43]:
first_midi = next(f for f in os.listdir(stimuli_folder) if f.endswith('.mid') or f.endswith('.midi'))
note_lengths = get_note_lengths(os.path.join(stimuli_folder, first_midi))

Found 88 notes in ../../stimuli/Gold\10_OrchestralSuiteNo2_excerpt2.mid


array([ 0, 49, 25, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 37, 13, 37, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13])

In [45]:
midi_data = {}
for filename in os.listdir(stimuli_folder):
    if filename.endswith('.mid') or filename.endswith('.midi'):
        match = re.search(r'^(\d+)', filename)
        if match:
            number = int(match.group(1))
            midi_path = os.path.join(stimuli_folder, filename)
            note_lengths = get_note_lengths(midi_path)
            midi_data[number] = note_lengths

Found 88 notes in ../../stimuli/Gold\10_OrchestralSuiteNo2_excerpt2.mid
Found 93 notes in ../../stimuli/Gold\11_18StudiesNo1.mid
Found 67 notes in ../../stimuli/Gold\12_5DivertimentosNo2_excerpt1.mid
Found 70 notes in ../../stimuli/Gold\13_Gavotte.mid
Found 145 notes in ../../stimuli/Gold\14_MaidenVoyage.mid
Found 84 notes in ../../stimuli/Gold\15_7VariationsTheme.mid
Found 50 notes in ../../stimuli/Gold\16_3FantasiestuckeNo1_excerpt1.mid
Found 67 notes in ../../stimuli/Gold\17_5DivertimentosNo2_excerpt2.mid
Found 89 notes in ../../stimuli/Gold\18_35ExercisesNo3.mid
Found 91 notes in ../../stimuli/Gold\19_18StudiesNo6_excerpt2.mid
Found 94 notes in ../../stimuli/Gold\1_StreamsOfKilnaspig.mid
Found 138 notes in ../../stimuli/Gold\20_Carmen.mid
Found 78 notes in ../../stimuli/Gold\21_OrchestralSuiteNo2_excerpt1.mid
Found 87 notes in ../../stimuli/Gold\22_35ExercisesNo15.mid
Found 72 notes in ../../stimuli/Gold\23_3FantasiestuckeNo1_excerpt2.mid
Found 91 notes in ../../stimuli/Gold\24_18S

In [49]:
midi_data

{10: array([ 0, 49, 25, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 37, 13, 37, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13]),
 11: array([  0,  13,  13,  13,  13, 121,  13,  13, 121, 121,  13,  13, 121,
        121,  13,  13, 121,  13,  13,  13,  13, 121,  13,  13, 121, 121,
         13,  13, 121, 121,  13,  13, 121,  13,  13,  13,  13,  13,  13,
         13, 121, 121,  13,  13, 121, 121,  13,  13, 121,  13,  13,  13,
         13, 121,  13,  13, 121, 121,  13,  13, 121, 121,  13,  13, 121,
         13,  13,  13,  13,  13,  13,  13,  13,  13,  13,  13,  13,  13,
         13,  13,  13,  13,  13,  13,  13,  13,  13,  13,  13,  13,  13,
         13,  13]),
 12: array([  0,  13,  13,  13,  13, 241,  13,  13,  13,  1

# Calculated duration weighted mean

In [53]:
weighted_gold = {}

for number, (ic, entropy) in gold_sorted:
    if number in midi_data:
        # for IC and Entropy, find corresponding note length array in midi note duration
        note_lengths = midi_data[number]
        
        assert len(ic) == len(entropy) == len(note_lengths)
        min_length = min(len(ic), len(entropy), len(note_lengths))
        ic = ic[:min_length]
        entropy = entropy[:min_length]
        note_lengths = note_lengths[:min_length]
        
        # total duration
        total_duration = np.sum(note_lengths)
        
        # weighted averages
        weighted_ic = np.sum(ic * note_lengths) / total_duration
        weighted_entropy = np.sum(entropy * note_lengths) / total_duration
        weighted_gold[number] = (weighted_ic, weighted_entropy)

# sort
duration_weighted_mean = sorted(weighted_gold.items())

In [54]:
duration_weighted_mean

[(1, (1.8661986811354205, 3.1911648242029496)),
 (2, (4.368538298440854, 5.7102545160589475)),
 (3, (4.8859366885666144, 4.198640462101855)),
 (4, (5.5226900818495785, 4.977703764244157)),
 (5, (4.050087721296653, 4.752884972718465)),
 (6, (3.198003979349279, 5.586292440918192)),
 (7, (5.080094165975521, 4.8356954308841695)),
 (8, (5.6159508099109345, 4.313228412993181)),
 (9, (4.103954309702266, 3.4683286068408985)),
 (10, (3.1691112624300293, 4.142892217074061)),
 (11, (4.54499617850136, 5.222702482010469)),
 (12, (8.037243020448145, 4.821915986040113)),
 (13, (4.563118560435888, 4.757457618102608)),
 (14, (7.086181559553674, 6.665397305918473)),
 (15, (5.980110047615312, 5.245866627066209)),
 (16, (8.19672424944242, 5.657480197400061)),
 (17, (8.73556654488781, 4.869843189985407)),
 (18, (4.889189571492089, 3.6212911653658146)),
 (19, (4.049216276119261, 4.726679024195035)),
 (20, (5.1604945698259765, 5.548690932606846)),
 (21, (5.775077679425761, 5.179856915577359)),
 (22, (3.60252

In [59]:
ppm_mDW_IC = np.array([item[1][0] for item in duration_weighted_mean])
ppm_mDW_Entropy = np.array([item[1][1] for item in duration_weighted_mean])

# Create a dictionary with the two arrays
ppm_mDW_mat = {
    'ppm_mDW_IC': ppm_mDW_IC.reshape(-1, 1),
    'ppm_mDW_Entropy': ppm_mDW_Entropy.reshape(-1, 1)
}

sio.savemat(output_file, ppm_mDW_mat)